In [10]:
import os
import re
import glob
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [11]:
from sklearn.ensemble import RandomForestClassifier


In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # Should print True if successful
print(torch.cuda.device_count())  # Number of GPUs recognized

True
1


In [13]:
def categorize_ad(ctr, impressions):
    """
    Returns an integer label:
      0 -> No success
      1 -> Moderately viral
      2 -> Very viral
    """
    # Current rules
    if impressions < 100_000 and ctr < 0.1:
        return 0  # "No success"
    elif impressions >= 300_000 or ctr >= 0.2:
        return 2  # "Very viral"
    else:
        return 1  # "Moderately viral"


In [14]:
class ViralAdsDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))
        
        # Regex to parse filenames like 0.12_100000_12.jpg
        #   group(1) = CTR, group(2) = impressions, group(3) = id
        self.pattern = re.compile(r"^([\d.]+)_([\d]+)_(.+)\.jpg$")
        
        # Store (path, label) for each image
        self.samples = []
        for path in self.image_paths:
            filename = os.path.basename(path)
            match = self.pattern.match(filename)
            if match:
                ctr_str = match.group(1)
                impressions_str = match.group(2)
                # _id = match.group(3)  # if you need the ad id, you can parse it here
                
                try:
                    ctr_value = float(ctr_str)
                    impressions_value = float(impressions_str)
                except:
                    # skip if parse fails
                    continue
                
                label = categorize_ad(ctr_value, impressions_value)
                self.samples.append((path, label))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [15]:
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = ViralAdsDataset(image_folder=r"C:\Bakalauras\filtered_images", 
                          transform=img_transform)

# Train-Test Split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [16]:
# Loading pretrained ResNet (ResNet18)
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = resnet.fc.in_features

# Replace final layer with a 3-class classification layer
resnet.fc = nn.Linear(num_ftrs, 3)

model = resnet


In [17]:
# 1) Pretrained ResNet without final layer
resnet_feature = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet_feature.fc = nn.Identity()
resnet_feature.eval()
resnet_feature.to(device)

# 2) Build embeddings dataset
all_embeddings = []
all_labels = []

for images, labels in DataLoader(dataset, batch_size=16, shuffle=False):
    images = images.to(device)
    with torch.no_grad():
        emb = resnet_feature(images)  # shape [batch_size, 512]
    all_embeddings.append(emb.cpu().numpy())
    all_labels.append(labels.numpy())

all_embeddings = np.concatenate(all_embeddings, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# 3) Train Test Split in sklearn style
X_train, X_val, y_train, y_val = train_test_split(all_embeddings, all_labels, 
                                                  test_size=0.2, random_state=42)

# 4) Random Forest
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# 5) Evaluate
y_pred = rfc.predict(X_val)
print(classification_report(y_val, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         4
           1     0.5455    0.1714    0.2609        35
           2     0.7360    0.9485    0.8288        97

    accuracy                         0.7206       136
   macro avg     0.4272    0.3733    0.3632       136
weighted avg     0.6653    0.7206    0.6583       136



c:\Users\vadim\.conda\envs\myenvTensor\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vadim\.conda\envs\myenvTensor\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vadim\.conda\envs\myenvTensor\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [18]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# y_true and y_pred are arrays of shape [num_samples]
# each entry is 0, 1, or 2 (the class)

mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

from sklearn.metrics import r2_score
r2 = r2_score(y_val, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print(f"R²: {r2:.4f}")

MAE: 0.3088235294117647
MSE: 0.36764705882352944
RMSE: 0.6063390625908325
R²: -0.3367
